<a href="https://colab.research.google.com/github/ShiroKealii/pruebas02/blob/main/c_villar_n_leva_Presentaci%C3%B3n_chat_bot_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar json
Crear archivo .json y el diccionario con el que "alimentaremos" a nuestro chat bot

Uso del archivo JSON:

Cargar información específica: El archivo JSON actúa como una base de datos que el chatbot puede consultar para obtener información detallada sobre las ciudades.

Facilidad de actualización: Los datos en el archivo JSON se pueden actualizar fácilmente sin necesidad de modificar el código principal del chatbot.



In [ ]:
import json

# Crear el archivo JSON directamente desde el código.
ciudades_data = {
    "ciudades": [
        {
            "nombre": "París",
            "transporte": ["El metro, autobuses, bicicletas y taxis son los principales medios de transporte."],
            "alojamiento": ["Puedes encontrar una amplia gama de alojamientos, desde hoteles de lujo hasta hostales y apartamentos."],
            "atracciones": ["Torre Eiffel, Museo del Louvre, Catedral de Notre-Dame, Campos Elíseos."]
        },
        {
            "nombre": "Roma",
            "transporte": ["El metro, autobuses y tranvías son los medios de transporte más comunes."],
            "alojamiento": ["Roma ofrece hoteles históricos, modernos apartamentos y acogedores bed & breakfast."],
            "atracciones": ["Coliseo, Foro Romano, Panteón, Fontana di Trevi, Vaticano."]
        },
        {
            "nombre": "Barcelona",
            "transporte": ["El metro, autobuses, tranvías y bicicletas son opciones populares."],
            "alojamiento": ["Desde hoteles boutique hasta hostales económicos y apartamentos turísticos."],
            "atracciones": ["Sagrada Familia, Parque Güell, La Rambla, Barrio Gótico."]
        },
        {
            "nombre": "Londres",
            "transporte": ["El metro, autobuses, trenes y bicicletas públicas son ampliamente utilizados."],
            "alojamiento": ["Gran variedad de hoteles, hostales y apartamentos disponibles."],
            "atracciones": ["Big Ben, London Eye, Museo Británico, Torre de Londres."]
        },
        {
            "nombre": "Berlín",
            "transporte": ["El sistema de trenes, autobuses y tranvías es muy eficiente."],
            "alojamiento": ["Hoteles modernos, hostales y apartamentos asequibles."],
            "atracciones": ["Puerta de Brandeburgo, Muro de Berlín, Museo de Pérgamo, Reichstag."]
        },
        {
            "nombre": "Ámsterdam",
            "transporte": ["Bicicletas, tranvías y autobuses son los principales medios de transporte."],
            "alojamiento": ["Hoteles con encanto, hostales y casas flotantes."],
            "atracciones": ["Museo Van Gogh, Casa de Ana Frank, Rijksmuseum, Canales de Ámsterdam."]
        },
        {
            "nombre": "Madrid",
            "transporte": ["Metro, autobuses y trenes de cercanías son muy usados."],
            "alojamiento": ["Hoteles de lujo, hostales y apartamentos céntricos."],
            "atracciones": ["Museo del Prado, Parque del Retiro, Palacio Real, Plaza Mayor."]
        }
    ]
}

# Guardar el JSON en un archivo
#Esto permite utilizar las funciones del módulo json para trabajar con datos en formato JSON.
with open('ciudades.json', 'w', encoding='utf-8') as f:
    json.dump(ciudades_data, f, ensure_ascii=False, indent=4)#Se abre un archivo llamado ciudades.json en modo de escritura ('w') con codificación UTF-8.
    #El with se usa para asegurar que el archivo se cierre correctamente después de escribir en él.


**json:** Biblioteca estándar de Python para trabajar con datos en formato JSON.

**transformers:** Biblioteca de Hugging Face que proporciona modelos de procesamiento de lenguaje natural (NLP).

**AutoTokenizer y AutoModelForCausalLM:** Clases de transformers que cargan el tokenizador y el modelo de lenguaje, respectivamente.

**set_seed:** Función para establecer una semilla aleatoria, lo que hace que los resultados sean reproducibles.

**model_name:** Nombre del modelo preentrenado en español que estamos usando.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed#Biblioteca de Hugging Face que proporciona modelos de procesamiento de lenguaje natural (NLP)
# Clases de transformers que cargan el tokenizador y el modelo de lenguaje
model_name = "flax-community/gpt-2-spanish"#modelo con el que trabajaremos en el caso de que no se realicen consultas especificas

# Cargar el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)#divide el texto
model = AutoModelForCausalLM.from_pretrained(model_name)#el modelo del lenguaje que vamos a utilizar

set_seed(42)  #( Esta función nos permitirá para establecer una semilla aleatoria,
#lo que hace que los resultados sean reproducibles.)

 - Cargar el archivo JSON

In [ ]:
# Cargar el archivo JSON
with open('ciudades.json', 'r', encoding='utf-8') as f:#Esta orden nos abrirá el archivo en modo lectura
    datos_ciudades = json.load(f)#Cargará el contenido del archivo .json


**open:** Abre el archivo ciudades.json en modo lectura.

**json.load(f):** Carga el contenido del archivo JSON en la variable datos_ciudades.

- Función para generar respuestas usando GPT-2

In [ ]:
def generar_respuesta(prompt, max_length=150):#Esta función generará respuesta usando el modelo GPT-2
    inputs = tokenizer.encode(prompt, return_tensors="pt")#Codifica la entrada del susuarios en tensores
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)#Genera texto basado en la entrada codificada
    return tokenizer.decode(outputs[0], skip_special_tokens=True)#decodifica el tensor de salida en texto


**generar_respuesta:** Función que genera una respuesta utilizando el modelo GPT-2.

**tokenizer.encode(prompt, return_tensors="pt"):** Codifica la entrada del usuario en tensores.

**model.generate(...):** Genera texto basado en la entrada codificada.

**tokenizer.decode(outputs[0], skip_special_tokens=True):** Decodifica el tensor de salida en texto.

- Función para buscar información en el archivo JSON

In [ ]:
def buscar_info_ciudad(ciudad, tipo_info):#Esta función busca información específica de las ciudades
    for entry in datos_ciudades["ciudades"]:#Itera sobre todas las ciudades
        if entry["nombre"].lower() == ciudad.lower():#Compara el nombre de la ciudad en minúsculas para asegurar una coíncidencia
            return entry.get(tipo_info, ["Lo siento, no tengo información sobre eso."])[0]#Obtiene la información solicitada o devuelve un mensaje de error si no se encuentra.
    return "Lo siento, no tengo información sobre esa ciudad."


**buscar_info_ciudad:** Función que busca información específica (transporte, alojamiento, atracciones) sobre una ciudad en el archivo JSON.

**for entry in datos_ciudades["ciudades"]:** Itera sobre todas las ciudades en el JSON.

**if entry["nombre"].lower() == ciudad.lower():** Compara el nombre de la ciudad en minúsculas para asegurar una coincidencia insensible a mayúsculas/minúsculas.

**entry.get(tipo_info, ["Lo siento, no tengo información sobre eso."])[0]:** Obtiene la información solicitada o devuelve un mensaje de error si no se encuentra.

- Función de chat para interactuar con el usuario

In [ ]:
def chat():#Con esta función interactuará con el usuario
    print("¡Hola! Soy tu asistente de viajes. ¿En qué puedo ayudarte hoy?")
    while True:
        user_input = input("Tú: ")#captura la entrada del usuario
        if user_input.lower() in ["salir", "adiós", "gracias"]:#Permite salir del chat
            print("Chatbot: ¡Hasta luego! ¡Espero haberte ayudado!")
            break

        # Procesar la entrada del usuario
        if "transporte" in user_input.lower():#En el caso de que el usuario pregunte por el transporte
            ciudad = user_input.split()[-1]#El nombre de la ciudad deberá aparecer al final de la consulta
            response = buscar_info_ciudad(ciudad, "transporte")#En esta linea llamamos a la función para obtener información sobre el transporte
        elif "alojamiento" in user_input.lower():#En el caso de que la consulta sea sobre el alojamiento
            ciudad = user_input.split()[-1]#El nombre de la ciudad deberá aparecer al final de la consulta
            response = buscar_info_ciudad(ciudad, "alojamiento")#Llamamos a la función para obtener información sobre el transporte
        elif "atracciones" in user_input.lower():#En el caso de qua la consulta sea sobre el Atracciones
            ciudad = user_input.split()[-1]#El nombre de la ciudad deberá aparecer al final de la consulta
            response = buscar_info_ciudad(ciudad, "atracciones")#Llamamos a la función para obtener información sobre el atracciones
        else:
            response = generar_respuesta("Responde en español: " + user_input)#se utiliza para generar una respuesta utilizando
            #el modelo de lenguaje GPT-2 en los casos en los que el usuario no está preguntando específicamente sobre transporte,
            #alojamiento o atracciones en una ciudad

        print(f"Chatbot: {response}")

# Iniciar el chat
chat()#función principal, maneja la iteración con el usuario


¡Hola! Soy tu asistente de viajes. ¿En qué puedo ayudarte hoy?
Tú: Madrid turismo


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: Responde en español: Madrid turismo, turismo de Madrid, Madrid ciudad, madrid turismo
El turismo español dona un total  ______ __________ euros a la Fundación para la Promoción del Turismo _____________________________ para el desarrollo.................................
La Fundación _________________________________________ __, ___, y la Asociación......................................................................, han suscrito un convenio 
con el fin  establecer un marco  marco para que las empresas turísticas puedan desarrollar su actividad  con el objetivo dd
d esde el año dc dl dt dp nte dn dm df db dh dea deh ie eo ipo 
Tú: Turismo de Roma


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: Responde en español: Turismo de Roma, Roma. La ciudad eslava dona a Roma su patrimonio histórico, artístico y cultural.
La ciudad italiana osa ser la capital urativa   del Imperio Romano. Su nombre proviene  por el nombre  que se le dio  a partir 
El nombre del país ʺItaliaʼ se debe a la ciudad que la vio nacer,  la cual  fue la primera ciudad del mundo  con el título  “Italia”
En el año  1899, el emperador Augusto, con la ayuda omo  el gobierno  y la fuerza  los militares, se fundó la República  Italiana. En el transcurso dl siglo XIX, la población
Tú: Turismo en Madrid


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: Responde en español: Turismo en Madrid.
El turismo de Madrid  iza la bandera  a la entrada del Palacio Real  el día  día del Rey Don Juan Carlos  y  del Príncipe Don Felipe 
La bandera del Reino  ondea  sobre el Palacio  desde el momento  que el Rey  ha sido recibido por el Presidente del Gobierno  Sr. D. Felipe VI  por la Reina Doña Sofía y el Príncipe  D Juan Manuel. El Rey ha recibido a los Reyes  con el saludo  saludo del Presidente  S. E. el Sr., el Jefe del Estado, el jefe dle Gobierno, y la presidenta del Congreso  Sra. Carme Chacón. La bandera ha onde
Tú: Salir
Chatbot: ¡Hasta luego! ¡Espero haberte ayudado!


**chat:** Función principal que maneja la interacción con el usuario.

**input("Tú: "):** Captura la entrada del usuario.

**if user_input.lower() in ["salir", "adiós", "gracias"]:** Permite salir del chat con ciertos comandos.

**if "transporte" in user_input.lower():** Detecta si el usuario pregunta sobre transporte.

**ciudad = user_input.split()[-1]:** Asume que la última palabra en la entrada del usuario es el nombre de la ciudad.

**response = buscar_info_ciudad(ciudad, "transporte"):** Llama a la función para obtener información sobre transporte en la ciudad especificada.

**else:** Si la entrada no contiene palabras clave específicas, utiliza GPT-2 para generar una respuesta general.




En este código se ha combinado un modelo de lenguaje avanzado (GPT-2) con datos estructurados (JSON) que aportará respuestas útiles y específicas sobre viajes.
Aprovecha la capacidad de los modelos de lenguaje natural para generar respuestas coherentes y contextuales.


 Proporciona información específica sobre transporte, alojamiento y atracciones en varias ciudades europeas.
Responde preguntas generales o proporcionar información adicional mediante generación de lenguaje natural.

Uso del archivo JSON:

Cargar información específica: El archivo JSON actúa como una base de datos que el chatbot puede consultar para obtener información detallada sobre las ciudades.
Facilidad de actualización: Los datos en el archivo JSON se pueden actualizar fácilmente sin necesidad de modificar el código principal del chatbot.

Con esta estructura, el chatbot puede ofrecer información precisa y detallada, al tiempo que mantiene la capacidad de responder a preguntas generales gracias al modelo GPT-2.